In [ ]:
import tensorflow as tf
import numpy as np
from PIL import Image
import glob

In [ ]:
tf.reset_default_graph()

# Set up the two queues: one for training and one for validation
train_files = glob.glob("/home/tom/data/notMNIST/notMNIST_small/A/S*.png")
train_q     = tf.train.string_input_producer(train_files, num_epochs=1)
vfiles      = tf.train.match_filenames_once("/home/tom/data/notMNIST/notMNIST_small/A/M*.png")
val_q       = tf.train.string_input_producer(vfiles, num_epochs=1)

In [ ]:
# Allow selection from either the training q or validation q, based on value of "is-training"
is_training = tf.placeholder(tf.bool, shape=None)
q_selector = tf.cond(is_training,
                     lambda: tf.constant(0),
                     lambda: tf.constant(1))
q = tf.QueueBase.from_list(q_selector, [train_q, val_q])

In [ ]:
# Read and decode the image
reader     = tf.WholeFileReader()
key, value = reader.read(q)

my_img = tf.image.decode_png(value)
my_img.set_shape((28,28,1))

image_batch = tf.train.shuffle_batch(
    [my_img],
    batch_size=5,
    capacity=2000,
    min_after_dequeue=10,
    allow_smaller_final_batch=True)

In [ ]:
init_all   = tf.global_variables_initializer()
init_local = tf.local_variables_initializer()

s = train_q.size

count = 0

with tf.Session() as sess:
    sess.run(init_all)
    sess.run(init_local)
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    with coord.stop_on_exception():
        while not coord.should_stop():
            print(sess.run(key, {is_training: True}))
            count+=1
    print("count: {}".format(count))
    #except tf.errors.OutOfRangeError:
        
    coord.request_stop()
    coord.join(threads)